In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import datetime
import nltk
import math
from sklearn import datasets, linear_model, metrics
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, scale
from sklearn.model_selection import cross_val_score, KFold, cross_val_predict
%matplotlib inline

In [44]:
# Part 1

hackernews_items = pd.read_csv('hn_items.csv',delimiter=',',encoding='latin-1')

nltk.download('vader_lexicon')

hackernews_items

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\piske\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,id,type,by,time,text,dead,parent,poll,kids,score,title,parts
0,1,story,pg,1160418111,NaN,NaN,NaN,NaN,487171;15;234509;454410;82729,61.0,Y Combinator,NaN
1,2,story,phyllis,1160418628,NaN,NaN,NaN,NaN,454411,16.0,A Student's Guide to Startups,NaN
2,3,story,phyllis,1160419233,NaN,NaN,NaN,NaN,454412;531602,7.0,Woz Interview: the early days of Apple,NaN
3,4,story,onebeerdave,1160419662,NaN,NaN,NaN,NaN,454413,5.0,NYC Developer Dilemma,NaN
4,5,story,perler,1160419864,NaN,NaN,NaN,NaN,454414,7.0,"Google, YouTube acquisition announcement could...",NaN
5,6,story,perler,1160420200,NaN,NaN,NaN,NaN,454415,4.0,Business Intelligence the Inkling Way: cool pr...,NaN
6,7,story,phyllis,1160420455,NaN,NaN,NaN,NaN,454416,5.0,Sevin Rosen Unfunds - why?,NaN
7,8,story,frobnicate,1160421459,NaN,NaN,NaN,NaN,454417,10.0,LikeBetter featured by BBC,NaN
8,9,story,askjigga,1160421542,NaN,NaN,NaN,NaN,454418,4.0,weekendr: social network for the weekend,NaN
9,10,story,frobnicate,1160421674,NaN,NaN,NaN,NaN,454419,3.0,PhotoShow: Broadcast Photos to Cable TV,NaN


In [75]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
model = SentimentIntensityAnalyzer()

hn_text = hackernews_items.dropna(subset=['text'], how='all')['text'].values

df = pd.DataFrame(columns=['text','neg','pos'])

data = []

for text in hn_text:
    score = model.polarity_scores(text)
    
    data.append({'text':text,'neg':score['neg'],'pos':score['pos']})

df = df.append(data)

df.head(5)

,text,neg,pos
0,&#34;the rising star of venture capital&#34; -...,0.0,0.000
1,Is there anywhere to eat on Sandhill Road?,0.0,0.000
2,It's kind of funny that Sevin Rosen is giving ...,0.0,0.218
3,"This is interesting, but the limitations becom...",0.0,0.149
4,Stay tuned...,0.0,0.000


In [71]:
print('5 most positive: ')
print(df.nlargest(5,'pos'))

print('5 most negative: ')
print(df.nlargest(5,'neg'))

5 most positive: 
                text  neg  pos
605             sure  0.0  1.0
999              ha!  0.0  1.0
1307      Beautiful.  0.0  1.0
1628  Great, thanks!  0.0  1.0
1902            True  0.0  1.0
5 most negative: 
             text  neg  pos
512          dupe  1.0  0.0
1797         spam  1.0  0.0
2962          No.  1.0  0.0
3008       dupe.   1.0  0.0
3014  desperation  1.0  0.0


In [228]:
# Part 2

negatives = df['neg']
positives = df['pos']
posts = hackernews_items.dropna(subset=['text'], how='all')['text']

In [231]:
from sklearn.cluster import KMeans
    
folds = KFold(n_splits=10)
kmeans = KMeans(n_clusters=10)

X = negatives.values
Y = positives.values
XY = np.stack((X,Y),axis=1)

kmeans.fit(XY)
    
clusters = kmeans.cluster_centers_
    
pred = kmeans.predict(clusters)

kmeans.score(XY)

-17.48963411065672

In [ ]:
# Part 3